In [1]:
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()

w.dbutils.library.restartPython()

In [2]:
from databricks.connect import DatabricksSession
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from datetime import datetime, date
import pandas as pd
import polars as pl

spark = DatabricksSession.builder.getOrCreate()

ppa_details_df = pd.read_csv("utils/ppa_details.csv")
ppa_details = spark.createDataFrame(ppa_details_df)

In [3]:
from utils.extract import extract_deal_info
from utils.transform import clean_and_aggregate_earnings

start_date = "2025-01-01"
end_date = "2030-06-30"

sim_complete_deal_info = extract_deal_info()

In [4]:
aurora_df = spark.table("exploration.default.earnings_aurora_scenarios")
aurora_earnings = clean_and_aggregate_earnings(
    df=aurora_df,
    deal_info_df=sim_complete_deal_info,
    curve_name="aurora",
    end_date=end_date,
    is_aurora=True,
    agg_by_month=True
)

combined_earnings = aurora_earnings

In [6]:
combined_earnings_df = pl.from_pandas(combined_earnings.toPandas())

In [ ]:
(
    combined_earnings_df.filter(
        pl.col("total_earnings").is_not_null() & (pl.col("year") >= 2025)
    )
    .filter(
        pl.col("curve") == "Central"
    )
    .with_columns(
        pl.when(pl.col("buy_sell") == "Buy")
        .then(pl.col("total_income"))
        .otherwise(-pl.col("total_cost"))
        .alias("floating_earnings"),
        pl.when(pl.col("buy_sell") == "Buy")
        .then(-pl.col("total_cost"))
        .otherwise(pl.col("total_income"))
        .alias("fixed_earnings"),
        pl.when(pl.col("deal_id") == 888_888)
        .then(pl.lit("Hypothetical"))
        .otherwise(pl.col("status"))
        .alias("status"),
    )
    .sort(
        ["year", "month", "group", "regionid", "deal_id"]
    )
    .select(
        pl.col("curve").alias("Curve Name"),
        pl.col("year").alias("Year"),
        pl.col("month").alias("Month"),
        pl.col("deal_id").alias("Deal ID"),
        pl.col("deal_name").alias("Deal Name"),
        pl.col("product_name").alias("Product Name"),
        pl.col("instrument").alias("Instrument"),
        pl.when(pl.col("status").is_in(["Approved", "Confirmed"]))
        .then(pl.lit("Contracted"))
        .otherwise(pl.lit("Hypothetical"))
        .alias("Status"),
        pl.col("buy_sell").alias("Buy/Sell"),
        pl.col("strategy").alias("Strategy"),
        pl.col("regionid").alias("Region ID"),
        pl.col("group").alias("Group"),
        pl.col("volume_mwh").alias("Volume (MWh)"),
        pl.col("fixed_earnings").alias("Fixed Earnings"),
        (pl.col("fixed_earnings") / pl.col("volume_mwh")).alias("VW Fixed Rate"),
        pl.col("floating_earnings").alias("Floating Earnings"),
        (pl.col("floating_earnings") / pl.col("volume_mwh")).alias("VW Floating Rate"),
        pl.col("twp").alias("TW Floating Price"),
        pl.col("total_earnings").alias("Total Earnings"),
    )
    .write_csv("aurora_central.csv")
)